# Selfmade memory based

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from scipy.spatial.distance import (
    correlation as orig_correlation,
    cosine as orig_cosine
)
from surprise import KNNWithMeans, Dataset, Reader
from ranx import Qrels, Run, evaluate

from typing import Union, Callable

## Task generation

In [2]:
np.random.seed(10)
r_width = 10
r_height = 500

R, group = make_blobs(
    n_samples=r_height,
    n_features=r_width,
    centers=5,
    random_state=10
)
R = np.round((R-R.min())*10/(R.max()-R.min())).astype(int)

# add bias for each object
bias = np.random.randint(-2,3, [R.shape[0], 1])
R = R + bias
# sometimes bias can lead to ratings
R = np.where(R<0, 0, R)
R = np.where(R>10, 10, R)
R[:10, :]

array([[ 5,  7,  2,  7,  4,  7,  3,  7,  3,  1],
       [ 8,  5,  7,  9,  6,  7,  8,  7,  7,  8],
       [ 3,  6,  1,  7,  2,  6,  1,  6,  2,  0],
       [ 6,  0,  6,  6,  4,  1,  3,  6,  2,  0],
       [ 8,  9,  2,  7,  9,  7,  8,  5,  9,  8],
       [ 8,  9,  4, 10,  5,  8,  5, 10,  6,  4],
       [ 7,  4,  8,  2,  1,  3,  1,  6,  0,  5],
       [ 3,  0,  2,  5,  3,  3,  4,  3,  5,  4],
       [ 7,  3,  8,  2,  1,  3,  1,  7,  1,  5],
       [ 7,  8,  1,  5,  7,  6,  7,  3,  9,  7]])

In [3]:
np.random.seed(10)
R_frame = pd.Series(
    R.ravel(),
    index = pd.MultiIndex.from_tuples(
        [
            (j,i) 
            for j in np.arange(R.shape[0]) 
            for i in np.arange(R.shape[1])
        ],
        names = ["object", "item"]
    ),
    name = "rank"
).to_frame()

R_frame["relevant"] = (R_frame["rank"] > 5).astype("int")
R_frame["random_predict"] = np.random.rand(R_frame.shape[0])
R_frame.sample(10)

,,rank,relevant,random_predict
object,item,,,
38,7,6,1,0.037330
126,4,6,1,0.120591
147,3,8,1,0.285638
215,2,3,0,0.204547
44,5,3,0,0.501746
264,3,10,1,0.790650
401,7,8,1,0.507431
233,0,6,1,0.794765
361,9,7,1,0.169455


In [4]:
R_fr_train, R_fr_test = train_test_split(
    R_frame, 
    train_size=0.8, 
    random_state=100
)

# preparing test/train samples representation as
# as user/item matrix
R_mat_train = pd.DataFrame(
    R_fr_train["rank"].unstack(),
    columns = R_frame.index.get_level_values(1).unique()
)
R_mat_test = pd.DataFrame(
    R_fr_test["rank"].unstack(),
    columns = R_frame.index.get_level_values(1).unique()
)

In [5]:
metrics = [
    "precision@3", 
    "recall@3", 
    "ndcg@3"
]
R_fr_test[["object_str", "item_str"]] = \
    R_fr_test.index.to_frame()[["object", "item"]].astype("str")

qrels = Qrels.from_df(
    df=R_fr_test.reset_index(),
    q_id_col="object_str", 
    doc_id_col="item_str",
    score_col="relevant"
)

random_run = Run.from_df(
    df=R_fr_test.reset_index(),
    q_id_col="object_str",
    doc_id_col="item_str",
    score_col="random_predict"
)

evaluate(
    qrels, 
    random_run, 
    metrics=metrics
)

{'precision@3': 0.311804008908686,
 'recall@3': 0.6076837416481069,
 'ndcg@3': 0.5401559043967112}

## surprise

In [6]:
reader = Reader(
    rating_scale=(
        R.min().min(), R.max().max()
    )
)
train_set = Dataset.load_from_df(
    df=R_fr_train["rank"].reset_index(), 
    reader=reader
).build_full_trainset()
algo = KNNWithMeans().fit(train_set)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [7]:
R_fr_test["surpr_predict"] = [
    algo.predict(uid=uid, iid=iid).est 
    for uid, iid in R_fr_test.index
]

In [8]:
mean_absolute_error(
    R_fr_test["rank"],
    R_fr_test["surpr_predict"]
)

0.6451729505731039

In [9]:
surprise_run = Run.from_df(
    df=R_fr_test.reset_index(),
    q_id_col="object_str",
    doc_id_col="item_str",
    score_col="surpr_predict"
)
evaluate(
    qrels, 
    surprise_run, 
    metrics=metrics
)

{'precision@3': 0.3266518188567186,
 'recall@3': 0.6303266518188567,
 'ndcg@3': 0.6346257036375941}

## Selfmade

### Similarity measure

**Note** Many sources use difference instead of similarity, but similarity is the inverse of difference, so you can search not for items that have the smallest difference, but for items that have the strongest similarity.

We need a method to estimate how close the objects are to each other. The following cell defines such a function - it implements Pearson's correlation coefficient, which can solve some problems related to the RecSys domain.

In [37]:
def correlation(
        a : np.ndarray,
        b : np.ndarray
        ) -> float:
    '''
    Pearson correlation coefficient modified
    for our requirements. In particular, empty 
    handling

    Parameters
    ----------
    a : (N,) np.ndarray
        input array;
    b : (N, ) np.ndarray
        input array;

    Returns
    ----------
    out : float
        The Pearson correlation coefficient is 
        computed using only the common items 
        for both arrays. If it's not possible 
        to compute the coefficient, it returns 0, 
        indicating neutral similarity.
        The coefficient is a number ranging from -1 to 1.
    '''
    cond = ~(np.isnan(a) | np.isnan(b))
    # in case if there are only two
    # observations it's impossible
    # to compute coorrelation coeficient
    # it's invalid case - so we return 
    # the biggest possible distance
    if sum(cond) <=1:
        return 0.

    sub_a = a[cond]
    sub_b = b[cond]
    
    variation_a = (sub_a - sub_a.mean())
    variation_b = (sub_b - sub_b.mean())

    # to compute pirson correlation coefficient
    # all variables should have some variation
    if (variation_a==0).all() or (variation_b==0).all():
        return 0.
    
    cov = (variation_a*variation_b).sum()
    return cov/np.sqrt(
        (variation_a**2).sum()*(variation_b**2).sum()
    )

Here are some cases where this function has been used and the result.

In [38]:
result = correlation(
    np.array([0,1,2,3,4]),
    np.array([5,6,7,8,9])
)
print("Total correlation -", result)
result = correlation(
    np.array([np.NaN, 1, 2, np.NaN]),
    np.array([10, 10, 20, np.NaN])
)
print("Total correlation with empty -", result)
result = correlation(
    np.array([1,1,1,1]),
    np.array([3,2,1,2])
)
print("Constant variable -", result)
result = correlation(
    np.array([np.NaN, 2, np.NaN, 3]),
    np.array([1, np.NaN, 10, np.NaN])
)
print("Not enough common elements -", result)

Total correlation - 1.0
Total correlation with empty - 1.0
Constant variable - 0.0
Not enough common elements - 0.0


In [28]:
def basic_prediction(
        collaboration : np.ndarray,
        similarities : np.ndarray
        ) -> np.ndarray:
    """
    Базововая функция для формирования предсказания.
    Использует формулу 
    \frac{(\sum_{i}x_{ij}-\overline{x_i})sim_i}{\sum_i{|sim_i|}}.
    Игры, для которых ни один пользователь из коллаборации 
    не сделал предсказания, будут иметь пропуски в результатах.
    Parameters
    ----------
    collaboration : np.ndarray (
        <количество пользователей>, 
        <количество игр>
        )
        матрица предпочтений коллаборации;
    relevances : np.ndarray (<количество игр>)
        предпочтения рассматривамого пользователя;
    similarities : np.ndarray (<количество наблюдений>)
        похожести элементов;
    
    Returns
    -------
    out : np.ndarray(<количество игр>)
        скоры наблюдаемых игр по которым их
        можно отсортировать по предпочтительности.
    """
    users_mean = np.nanmean(collaboration, axis=1, keepdims=1)
    weighed_collab = (collaboration - users_mean)*similarities[:, np.newaxis]
    res = np.nansum(weighed_collab, axis=0)/np.abs(similarities).sum()
    
    # те игры в которые не поиграл не один пользователь
    # из коллаборации можно смело заменять на nan
    is_empty = np.isnan(collaboration).all(axis=0)
    res[is_empty] = np.NaN

    return res


class CollaborativeFilter:
    """
    Класс для выполнения коллаборативной фильтрации 
    на основе ближайших соседей.

    Attributes
    ----------
    similarity : Callable[[np.ndarray, np.ndarray], float]
        Фунция для вычисления похожести между двумя
        объектами;
    prediction : Callable[[np.ndarray, np.ndarray]
        Пороговое значение меры близости. Т.е. объекты у которых
        значение дистации выше не пропускаются;
    sim_threshold : float = -np.inf
        Пороговое отсечение похожести, ниже которого
        элементы не допусткаются в колаборации;
    n_nearest : int = 1000
        Максимальное количество наблюдений которые могут
        быть взяты в коллаборацию.
    """
    def __init__(
            self, 
            similarity : Callable[[np.ndarray, np.ndarray], float] = correlation,
            prediction : Callable[[np.ndarray, np.ndarray], float] = basic_prediction,
            sim_threshold : float = -np.inf,
            n_nearest : int = 1000
            ):
        self.similarity = similarity
        self.prediction = prediction
        self.sim_threshold = sim_threshold
        self.n_nearest = n_nearest
        
    
    def fit(self, X:Union[np.ndarray, pd.DataFrame]):
        '''
        Запомнить обучающую выборку.

        Parameters
        ----------
        X : np.ndarray (<количество клиентов>, <количество игр>)
            Наблюдения за решевантностями пользователей;

        Returns
        -------
        out : CollaborativeFilter
            сам объект модели.
        '''
        self.X=np.array(X)
        return self

    def get_similarities(self, X:np.ndarray):
        '''
        Получить предсказания расстояния до объекта
        для заданного множества векторов предпочтений
        пользователей.

        Parameters
        ----------
        X : np.ndarray(<количетсво пользователей>, <количество игр>)
            матрица, которая описывает предпочтения
            пользователей для которых надо сформировать
            предсказание;
        
        Returns
        -------
        out : np.ndarray (
            <количетсво сохраненных пользователей>, 
            <количество пользователей для предсказания>
            )
            нассив где каждый элемент это дистанция
            между i-м пользователем сохраненного набора
            данных и j-м пользователем набора данных
            для которого ведется предсказание.
        '''
        return np.apply_along_axis(
            func1d=lambda history_row: np.apply_along_axis(
                func1d=(
                    lambda predict_row: 
                    self.similarity(history_row, predict_row)
                ),
                arr=X, axis=1
            ), 
            arr=self.X, axis=1
        )
    

    def get_collaborations(self, X:np.ndarray):
        '''
        Получить коллаборации для заданного набора наблюдений.
        '''
        similarities = self.get_similarities(X=X)
        # перебираем столбики с похожестями пользователей
        # для которых ведется предсказание и достаем нужные 
        # коллаборации
        return np.apply_along_axis(
            func1d=lambda user_sim: (
                self.X[user_sim>self.sim_threshold,:][:self.n_nearest,:]
            ),
            axis=0, arr=similarities
        )


    def predict(self, X:np.ndarray)->np.ndarray:
        '''
        Получить предсказания для 
        пользователей с заданными предпочтениями

        Parameters
        ----------
        X : np.ndarray (<количетсво пользователей>, <количество игр>)
            матрица, которая описывает предпочтения
            пользователей для которых надо сформировать
            предсказание;
        
        Returns
        -------
        out : np.ndarray
            np.ndarray (<количество полльзователей>, <количество игр>)
            предсказания для заданных пользователей.
        '''
        if X.shape[1] != self.X.shape[1]:
            raise ValueError(
                "Количества игр в обучающем наборе данных "
                "и наборе для предсказания не совпадают."
                )
        
        similarities = self.get_similarities(X)